# Exposition au Covid-19 et offre de travail de long terme : 
## *Analyse de l'évolution des arrêts maladies par secteur d'activité et catégorie socioprofessionnelle à partir de l’Enquête Emploi*


Projet réalisé en 2025 par Lucile Aubain, Jean Lavallée et Paul Hobeika dans le cadre du cours de *Python pour la data science* enseigné en deuxième année de l'ENSAE.

<a id="introduction"></a>

## Introduction

La pandémie de Covid-19 a contaminé près de 800 millions de personnes et entraîné environ sept millions de décès dans le monde entre 2020 et 2025, dont 150 000 en France. Au-delà de ces conséquences sanitaires directes, la pandémie a engendré des effets durables sur les trajectoires individuelles et sur le fonctionnement du marché du travail. Ce projet s'intéresse à une dimension encore peu étudiée en France sous l'angle de l'économie de la santé et du travail : la hausse persistante des arrêts de travail pour maladie depuis 2020 et ses implications pour l'offre de travail de long terme.

La littérature médicale documente désormais largement l'existence de séquelles durables chez une fraction non négligeable des personnes infectées par le Covid-19, y compris après des formes initialement bénignes. Ces affections de long terme, regroupées sous l'appellation de « Covid long », recouvrent des symptômes variés – fatigue chronique, troubles cognitifs ou neurologiques, difficultés respiratoires – susceptibles de limiter durablement la capacité à exercer une activité professionnelle ([Groff et al., 2021](https://pubmed.ncbi.nlm.nih.gov/34643720/), [Ford et al., 2025](https://onlinelibrary.wiley.com/doi/abs/10.1002/ajim.70014?casa_token=NAKQSBiFLa8AAAAA%3AIUjd4EXrl4jU1-7FkTjwP6W7jjW2aly-mTo0LDxL27Oh4bVbCj11uAcFBwdYeCaNNGGWgzkhD-0v5TfP), [MacEwan et al., 2025](https://link.springer.com/article/10.1007/s11606-024-09062-5)).

Ces constats ont nourri une littérature économique récente qui analyse le rôle du Covid long dans la contraction de l'offre de travail observée après la pandémie. À partir de données américaines, Goda et al. ([2023](https://www.sciencedirect.com/science/article/pii/S0047272723000713)) montrent que les travailleurs absents au moins une semaine en 2020 pour raisons liées au Covid-19 ont une probabilité inférieure de 7 % d'être toujours actifs un an plus tard, contribuant à un déficit estimé à environ 500 000 emplois aux États-Unis. D'autres travaux mettent également en évidence des sorties durables de la population active associées au Covid long ([Evans et al., 2021](https://www.thelancet.com/journals/lanres/article/PIIS2213-2600\(21\)00383-0/fulltext?uuid=uuid%3A4cfb08b5-99b4-49e0-ab9b-14f97739bbb6), [Ziauddeen et al., 2022](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0264331)). À partir de données néerlandaises, Bussink et al. ([2025](https://pubmed.ncbi.nlm.nih.gov/41180946/)) estiment qu'une infection au Covid-19 réduit d'environ 1 point de pourcentage la probabilité d'emploi des nouveaux entrants sur le marché du travail dans les mois suivant la diplomation, un effet plus limité que dans les études portant sur l'ensemble de la population adulte, possiblement en raison d'une moindre prévalence du Covid long chez les jeunes.

Cette littérature s'inscrit plus largement dans les débats relatifs aux « travailleurs manquants » (*missing workers*) observés dans de nombreuses économies avancées après la pandémie. Au Royaume-Uni, Reuschke et Houston ([2022](https://www.tandfonline.com/doi/full/10.1080/13504851.2022.2098239)) estiment qu'environ 80 000 personnes ont quitté leur emploi en raison du Covid long, contribuant aux pénuries de main-d'œuvre. Ces résultats suggèrent que les effets macroéconomiques de la pandémie pourraient être sensiblement sous-estimés si l'on ne tient pas compte de ses conséquences sanitaires de long terme.

Plusieurs institutions ont ainsi avancé des ordres de grandeur importants quant à l'impact du Covid long sur l'offre de travail. Un rapport de la *Brookings Institution* ([2022](https://www.brookings.edu/articles/is-long-covid-worsening-the-labor-shortage/)) estime que jusqu'à quatre millions d'Américains pourraient être éloignés de l'emploi en raison du Covid long, soit environ 15 % des postes vacants en 2022. Dans l'Union européenne, la Commission européenne ([2024](https://economy-finance.ec.europa.eu/document/download/36713cbb-6cbf-4ddb-8a15-55a4f456e2cb_en?filename=eb077_en.pdf)) évalue la perte d'offre de travail associée au Covid long entre 0,2 et 0,3 % en 2021 et entre 0,3 et 0,5 % en 2022, correspondant à 600 000 à 1 million de personnes. Des estimations comparables sont proposées pour l'ensemble des pays de l'OCDE ([Abraham et Rendell, 2023](https://www.brookings.edu/wp-content/uploads/2023/03/BPEA_Spring2023_Abraham-Rendell_unembargoed.pdf), [Espinosa Gonzalez et Suzuki, 2024](https://www.oecd.org/content/dam/oecd/en/publications/reports/2024/06/the-impacts-of-long-covid-across-oecd-countries_f662b21c/8bd08383-en.pdf)). À ce titre, le Covid long pourrait porter le coût économique total de la pandémie à près de 1 000 milliards de dollars pour l'OCDE ([Espinosa Gonzalez et Suzuki, 2024](https://www.oecd.org/content/dam/oecd/en/publications/reports/2024/06/the-impacts-of-long-covid-across-oecd-countries_f662b21c/8bd08383-en.pdf)).

Dans ce contexte, ce travail étudie l'effet de l'exposition au Covid-19 en 2020 sur l'offre de travail de long terme des salariés en France, à partir des données de l'Enquête emploi en continu (EEC). En l'absence d'une mesure directe de l'exposition individuelle au virus dans l'enquête, nous proposons une méthode indirecte permettant de classer les secteurs et catégories socioprofessionnelles selon leur degré d'exposition en 2020. Nous faisons l'hypothèse que le taux d'arrêts de travail pour maladie observé en 2020 constitue un *proxy* pertinent de l'exposition au Covid-19 pour une catégorie donnée. Le contexte de crise sanitaire, marqué par l'arrêt de la production dans certains secteurs et par les mesures de confinement, a en effet conduit à une surexposition de certaines professions, tandis que d'autres en étaient relativement protégées ([Dubost, Pollak et Rey, 2020](https://drees.solidarites-sante.gouv.fr/sites/default/files/2020-10/DD62.pdf)).

Nos résultats montrent que les salariés des secteurs les plus exposés en 2020 réduisent durablement leur offre de travail, bien au-delà du pic de la pandémie. Une analyse en doubles-différences indique que les travailleurs des secteurs fortement exposés ont une probabilité supérieure d'environ 0,9 point de pourcentage d'être en arrêt maladie un an plus tard, relativement à des travailleurs comparables des secteurs peu exposés. Au regard de la moyenne globale de l'échantillon à 2,8 % pour la variable `sh_arret`, l'effet estimé représente une augmentation relative d'environ un tiers du recours aux arrêts maladie par rapport à la moyenne de l'échantillon. Cette hausse massive indique que l'impact de la pandémie n'est pas marginal, qu'il a au contraire profondément modifié le niveau de participation au travail dans les secteurs exposés.

Notre analyse en étude d'évènement permet en outre de valider notre hypothèse de tendances parallèles (*pre-trends*) et de mettre en évidence l'impact dynamique et la persistance de l'effet lié à l'exposition au Covid-19 en 2020. L'effet de l'exposition au Covid-19 sur les arrêts de travail atteint son maximum en 2021 avec un coefficient avoisinant 1,4 point de pourcentage, soit une hausse relative de 50 % du taux d'arrêt de travail au sein des secteurs traités par rapport au groupe de référence. Ce travail contribue ainsi à la littérature existante en apportant des éléments nouveaux pour le cas français, encore peu documenté, et en mettant en évidence un canal sanitaire susceptible d'expliquer une partie des difficultés de recrutement observées dans certains secteurs à partir de 2021.

Ce travail est organisé de la façon suivante. La section [Installation](#installation) présente les librairies et fonctions utilisées. La section [Données](#donnees) décrit les données et leur préparation, avant la construction d'un indicateur d'exposition sectorielle dans la section [Exposition au Covid-19 en 2020](#exposition). La section [Covid long et réduction de l'offre de travail](#modelisation) analyse les effets de long terme de l'exposition au Covid-19 sur l'offre de travail. La section [Conclusion](#conclusion) résume nos résultats et discute des limites de leur validité.

<a id="sommaire"></a>

### Sommaire

- [Introduction](#introduction)
- [Installation](#installation)
- [Données](#donnees)
- [Exposition au Covid-19 en 2020](#exposition)
    - [Les absences pour congé maladie comme proxy de l'exposition au Covid-19](#absences)
    - [Des inégalités structurelles dans la probabilité d'être en arrêt maladie](#inegalites)
    - [Calcul d'un score d'exposition par catégorie](#exposition)
    - [Analyse des déterminants de l'exposition au Covid-19](#regression)
- [Covid long et réduction de l'offre de travail](#modelisation)
    - [Doubles-différences](#doubles_differences)
    - [Étude d'évènement](#event_study)
- [Conclusion](#conclusion)

<a id="installation"></a>

## Installation

In [51]:
%%capture
%pip install -r scripts/requirements.txt

# Modules
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import requests
from io import BytesIO
from zipfile import ZipFile
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from lets_plot import *
from urllib import request
from io import BytesIO
import linearmodels as lm
from linearmodels.panel import PanelOLS
import seaborn as sns
import importlib
import statsmodels.formula.api as smf
import statsmodels.api as sm
from patsy.contrasts import Treatment


# Fonctions
from scripts import import_eec
from scripts import import_eec_all
from scripts import hospitalisations
from scripts import exposition
from scripts import dids

<a id="donnees"></a>

## Données

Notre analyse empirique repose sur l'exploitation des microdonnées de l'Enquête Emploi en Continu (EEC) de l'Insee. Nous identifions les situations d'absence au travail en isolant les individus en emploi n'ayant effectué aucune heure de travail durant la semaine de référence. Le motif de l'absence est identifié grâce à la variable renseignant la raison principale de l'absence (variable `RABS`). Nous restreignons notre champ d'analyse aux absences pour raisons de santé, définies par la modalité « Congé maladie ou accident du travail » (`RABS = 2`).

Cette variable est présente dans les fichiers détails de l'EEC, accessibles en *open data* directement depuis le site de l'Insee (mais pas depuis la librairie python `pynsee`). Les fichiers étant publiés chaque année, la base de données a été constituée par l'assemblage des vagues annuelles couvrant la période 2010-2024.

La construction de ce panel a nécessité un travail d'harmonisation des variables afin de corriger les changements de nomenclature et de codification intervenus au cours du temps. À titre d'illustration, la discrétisation et l'intitulé de la variable relative à l'âge ont été modifiés en 2020 (passage de AGE5 à AGE6). La plupart des variables qui nous intéressent restent cependant inchangées entre 2010 et 2024, en particulier : 

- la variable indiquant le statut d'activité des enquêtés (`ACTEU`), permettant de distinguer les actifs en emploi (`ACTEU = '1'`) du reste de la population.
- la variable permettant d'identifier les actifs déclarant n'avoir pas travaillé durant la semaine de référence pour cause de maladie (`RABS = '2'`).
- l'année de l'enquête (`ANNEE`).

D'autre variables dont nous avons besoin changent de nom (mais pas de nomenclature) selon les années. Lors de l'importation des données, nous renommons donc les variables suivantes afin de pouvoir les conserver dans la table résultant de la concaténation de toutes les fichiers détail de 2010 à 2024 : 

- la variable de catégorie socioprofessionnelle à deux chiffres : présente sous des noms différents, nous la renommons `CSE` pour l'ensemble des années
- la variable de secteur d'activité à 38 modalités : `NAFG38UN`

La fonction `load_eec_all()` permet de reconstruire la base pluriannuelle à partir des fichiers sources annuels. Cette opération inclut le traitement des formats historiques (fichiers `.dbf` de 2010 à 2017) et peut s'avérer coûteuse en temps de calcul.

Pour optimiser l'exécution, une version pré-traitée de la base a été sauvegardée au format `Parquet`. Le script est configuré par défaut pour charger cette version rapide.

- Pour rejouer la construction depuis les données brutes : décommenter l'appel à `import_eec_all.load_eec_all()` et commenter la dernière ligne `eec_all = pd.read_parquet("https://minio.lab.sspcloud.fr/phobeika/open_eec/eec_all.parquet")` ;
- Pour utiliser la version optimisée (recommandé) : conserver le chargement via pd.read_parquet.

In [52]:
# Import des données
 
# Méthode 1 (environ cinq minutes) : téléchargement direct depuis le 
# site de l'Insee ou via les url de secours en cas d'erreur
#
# eec_all = import_eec_all.load_eec_all()
#
# Sauvegarde locale pour usage ultérieur
#
# object_cols = eec_all.select_dtypes(include='object').columns
# eec_all[object_cols] = eec_all[object_cols].astype(str)
# eec_all.to_parquet('data/eec_all.parquet', index=False)



# Méthode 2 (trois secondes) : chargement depuis un fichier obtenu via la méthode 1
# sauvegardé sur l'espace de stockage du SSP Cloud

eec_all = pd.read_parquet("https://minio.lab.sspcloud.fr/phobeika/open_eec/eec_all.parquet")

Selon les années, les données importées n'ont pas exactement le même format. Pour éviter les erreurs liées à des petites différences entre les années (par exemple des modalités `1` et `1.0` pour des variables catégorielles qui ont été à un moment enregistrées comme valeurs numériques), nous utilisons la fonction `recodages()` qui permet d'éviter ces problèmes en plus que de distinguer les variables selon leur type (catégoriel ou numérique).

In [ ]:
# Recodages
eec_all = import_eec.recodages(
    eec_all, 
    vars_cat=['RABS', 'ACTEU', 'SEXE', 'CSE', 'PUB3FP', 'NAFG036UN'], 
    vars_num=['ANNEE']
    )

Enfin, le code suivant filtre l'ensemble des données de manière à ne travailler que sur la population active et ajoute des variables ` labels` pour rendre les modalités lisibles dans les tableaux et graphiques.

In [ ]:
# Filter pour ne conserver que les actifs en emploi
eec_actifs = eec_all[eec_all['ACTEU'] == "1"].copy()

# Ajout de variables 'labels' pour rendre les modalités lisibles
eec_actifs = import_eec.add_labels(eec_actifs)

<a id="exposition"></a>

## Exposition au Covid-19 en 2020

Cette section vise à construire une variable de score d'exposition au Covid-19 en 2020 à partir des données sur les arrêts de travail. Cette utilisation ne va pas de soi, car nous voulons ensuite mesurer l'effet de ce score sur les arrêts de travail pour maladie dans les années qui suivent. Il s'agit donc d'utiliser les arrêts de travail à la fois comme variable explicative et comme variable d'intérêt. Pour justifier la pertinence de cette méthode, nous faisons une double hypothèse. D'abord, nous faisons l'hypothèse que la pandémie de Covid-19 a été la cause principale d'arrêts pour maladie en 2020. Ensuite, nous supposons que même si le virus a continué à circuler les années suivantes, son effet sur la santé des actifs a été à la fois moindre (notamment en raison de la vaccination) et davantage "concurrencé" par d'autres maladies, si bien que la dynamique épidémique du Covid-19 est moins prédictive des arrêts pour maladie après 2020.

Cette section vise à justifier ces deux hypothèses et à construire la variable d'exposition que nous utiliserons par la suite pour rendre compte des différences post-2020. Pour cela, nous commençons par étudier la corrélation entre les arrêts de travail et les données d'hospitalisation liées au Covid-19. Nous mettons ensuite en évidence la variabilité des arrêts de travail dans le temps selon nos variables d'intérêt, variabilité grâce à laquelle nous construisons un indicateur en cinq modalités reflétant le degré d'exposition au Covid-19 en 2020. Enfin, nous montrons que les différences observées plus tôt se retrouvent en partie dans la probabilité d'être exposé au Covid en 2020. 

<a id="absences"></a>

### Les absences pour congé maladie en 2020 comme proxy de la contamination par le Covid-19

Nous faisons l'hypothèse que, dans le contexte pandémique de l'année 2020, les absences au travail pour maladie sont fortement corrélées à la circulation du Covid-19. Pour vérifier cette hypothèse, nous étudions ici la corrélation entre l'intensité pandémique mesurée à partir du nombre d'hospitalisations pour Covid-19 et les arrêts de travail pour maladie. Après avoir récupéré les données des hospitalisations liées au Covid-19 en 2020 via une API de Santé publique France, nous comparons l'évolution du nombre d'hospitalisations et d'arrêts maladies.

Tout d'abord, pour récupérer les données concernant le nombre d'hospitalisations liées au Covid-19, nous utilisons l'API Huwise de Santé publique France, présente sur le portail [open data Odissé](https://odisse.santepubliquefrance.fr/explore/dataset/covid-19-synthese-des-indicateurs-de-suivi-de-la-pandemie-dep/api/?sort=date). Après une lecture extensive de la [documentation de l'API](https://help.opendatasoft.com/apis/ods-explore-v2/) afin de contourner la faible limite de résultats par requête, nous exportons l'ensemble de la base "Covid-19 - Synthèse des indicateurs de suivi de la pandémie", *via* les commandes indiquées dans la documentation (regroupée dans la fonction `hospitalisations()`).

In [ ]:
# Importation des données de Santé Publique France
hosp = hospitalisations.import_hosp(
    url_main="https://odisse.santepubliquefrance.fr/api/explore/v2.1/catalog/datasets/covid-19-synthese-des-indicateurs-de-suivi-de-la-pandemie-dep/exports/json",
    url_backup="https://minio.lab.sspcloud.fr/phobeika/open_eec/covid-19-synthese-des-indicateurs-de-suivi-de-la-pandemie-dep.json"
)

Pour pouvoir grouper et sommer les observations par date, nous sélectionnons les données de 2020 en s'assurant qu'il existe toujours le même nombre de départements observés pour chaque date (101). Le nombre d'hospitalisations en fonction du temps est représenté graphiquement ci-dessous.

In [ ]:
hospitalisations.plot_hosp_year(hosp, year = 2020)

On retrouve bien les deux "vagues" caractéristiques du Covid en France en 2020. Il reste toutefois à savoir si les arrêts maladie suivent la même tendance que ces données d'hospitalisation. Comme l'échelle temporelle la plus fine contenue dans les fichiers détail des enquêtes emploi est le trimestre (plutôt que la journée pour les données d'hospitalisation), nous utilisons les quatre trimestres de 2020 pour comparer l'évolution des deux variables. Le résultat est présenté dans le graphique suivant.

In [ ]:
hospitalisations.plot_hosp_arrets_trim(eec_actifs, hosp, year=2020)

La corrélation entre le nombre d'arrêts de travail pour congé maladie et le nombre d'hospitalisations liées au Covid en 2020 est de 0.51, ce qui est satisfaisant pour un *proxy*, d'autant plus qu'une partie des personnes qui se seraient peut-être mises en arrêt de travail étaient déja arrêtées (par exemple parce qu'elles étaient au chômage partiel...), réduisant les variations du nombre d'arrêts de travail.

Il reste enfin à vérifier que la proportion d'arrêts de travail pour cause de maladie n'est plus corrélée avec les hospitalisations liées au Covid-19 pour les années après 2020. Nous reproduisons donc l'exercice ci-dessus pour les années 2021, 2022 et 2023, dernière année pour laquelle les donnés d'hospitalisation sont disponibles.

In [ ]:
hospitalisations.plot_hosp_arrets_trim(eec_actifs, hosp, year = 2021)
hospitalisations.plot_hosp_arrets_trim(eec_actifs, hosp, year = 2022)
hospitalisations.plot_hosp_arrets_trim(eec_actifs, hosp, year = 2023)

La corrélation positive observée en 2020 entre les deux variables disparaît dans les années qui suivent. Cela semble ainsi valider notre seconde hypothèse.

On pourrait néanmoins discuter de la pertinence d'utiliser le nombre d'hospitalisations comme mesure de l'intensité de l'épidémie de Covid-19, car les campagnes de vaccination ont largement réduit les formes graves du virus, sans nécessairement réduire de la même proportion sa transmission et ses effets sur les arrêts de travail. On se limitera néanmoins à cette variable dans le cadre de ce projet, car l'échelle temporelle du trimestre limite quoi qu'il arrive la portée de cette analyse de corrélation. Pour aller plus loin, nous pourrions avoir recours aux données de l'enquête emploi diffusées par [Progedo](https://www.progedo.fr/la-serie-des-enquetes-emploi/) qui permettent d'identifier la semaine de référence, que l'on pourrait mettre en parallèle avec la mesure de la concentration en virus dans les eaux usées (données [Sum'eaux](https://www.data.gouv.fr/datasets/surveillance-du-sars-cov-2-dans-les-eaux-usees-sumeau)).

<a id="inegalites"></a>

### Des inégalités structurelles dans la probabilité d'être en arrêt maladie

Si la variable d'arrêt de travail constitue un *proxy* de l'exposition au Covid-19 en 2020, elle ne peut s'utiliser qu'à un niveau agrégé. En effet, les données ne permettent pas de suivre des individus dans le temps, si bien qu'il n'est pas possible d'identifier un effet d'être en arrêt de travail en 2020 sur les arrêts de travail ultérieurs à une échelle individuelle. Il est toutefois possible d'agréger les données selon des catégories plus ou moins exposées au virus en 2020. On vérifie ici que les variables de catégorie socioprofessionnelle et de secteur d'activité permettent bien de distinguer différentes catégories d'actifs plus ou moins exposées.

In [ ]:
# Calcul de la proportion d'actifs en arrêt de travail selon les années en fonction de la PCS
df_pcs_annee = exposition.exposition_annee(eec_actifs, var="CSER_label", year_col="ANNEE")

# Représentation graphique de l'évolution de la proportion d'arrêts maladie par catégorie socioprofessionnelle
exposition.plot_evolution_proportion(
    df=df_pcs_annee,
    year_col="ANNEE",
    group_col="CSER_label",
    value_col="proportion_rabs2",
    title="Évolution de la proportion d'absences au travail pour congé maladie selon la PCS",
    ylabel="Pourcentage d'actifs absents du travail pour congé maladie"
    )

Les catégories socioprofessionnelles agrégées permettent bien de distinguer les actifs selon le pourcentage d'arrêt de travail parmi chaque catégorie. De manière générale, les groupes d'ouvriers et d'employés sont ceux pour lesquels la proportion d'actifs en arrêt est la plus forte (entre 4 et 6 %), tandis que les catégories d'indépendants et les cadres sont en arrêt moins fréquemment (entre 1 et 2 % sur toute la période). Les professions intermédiaires se situent entre ces deux groupes (entre 2 et 3,5 %). 

Les différences structurelles entre catégories ne sont toutefois pas ce qui nous intéresse ici, puisque nous cherchons à estimer des différences d'*exposition* au Covid-19 en 2020. Comme nous l'expliquerons par la suite, cet effet correspond plutôt à l'*accroissement différentiel* de la proportion d'actifs en arrêt entre 2019 et 2020. On remarque que cet accroissement est plus le plus fort pour les catégories d'employés et les professions intermédiaires, pour lesquelles il représente environ 1 point de pourcentage. Il est un peu plus faible pour les ouvriers, et beaucoup plus faible pour les autres catégories d'actifs.

In [ ]:
# Calcul de la proportion d'actifs en arrêt de travail selon les années en fonction de NAF agrégée
df_naf_annee = exposition.exposition_annee(eec_actifs, var="NAFR", year_col="ANNEE")

# Représentation graphique de l'évolution de la proportion d'arrêts maladie par secteur d'activité
exposition.plot_evolution_proportion(
    df=df_naf_annee,
    year_col="ANNEE",
    group_col="NAFR",
    value_col="proportion_rabs2",
    title="Évolution de la proportion d'absences au travail pour congé maladie selon le secteur d'activité",
    ylabel="Pourcentage d'actifs absents au travail pour congé maladie",
    )

Ce graphique est peu lisible, car la variable de secteur d'activité NAF comporte trop de catégories pour représenter leurs évolutions aisément, même à un niveau agrégé. On peut toutefois remarquer que la proportion d'actifs en arrêt pour maladie augmente davantage pour certaines catégories, par exemple pour les catégories `NAF == E` qui correspond au secteur de la distribution d'eau et au traitement des déchets, ou `NAF == Q` qui regroupe les professions du secteur de la santé et de l'action sociale.

Enfin, l'évolution des arrêts maladie est également variable selon le sexe, comme le montre le graphique suivant :

In [ ]:
# Calcul de la proportion d'hommes et de femmes en arrêt de travail selon les années 
df_sexe_annee = exposition.exposition_annee(eec_actifs, var="SEXE_label", year_col="ANNEE")

# L'évolution du nombre d'arrêts de travail pour congé maladie en fonction du sexe
exposition.plot_evolution_proportion(
    df=df_sexe_annee,
    year_col="ANNEE",
    group_col="SEXE_label",
    value_col="proportion_rabs2",
    title="Évolution de la proportion d'absences au travail pour congé maladie selon le sexe",
    ylabel="Pourcentage d'actifs absents au travail pour congé maladie",
    colors={"Homme":"#7A57FF", "Femme":"#C869FF"}
)

Les femmes sont en moyenne davantage en arrêt que les hommes, et la proportion d'arrêt a également augmenté davantage en 2020 pour les femmes que pour les hommes. On pourrait donc envisager d'utiliser la variable de sexe comme une variable explicative, notamment car les recherches médicales semblent montrer que les femmes développent davantage des formes de Covid long que les hommes. Cela aurait toutefois l'inconvénient de masquer les effets structurels de l'exposition différenciée au Covid-19 selon le sexe en 2020, comme le suggèrent les deux graphiques précédents : les femmes sont en effet davantage représentées parmi les secteurs les plus exposés, ce que l'on montre dans la section suivante.

<a id="exposition"></a>

### Calcul d'un score d'exposition par catégorie

Les analyses qui précèdent nous fournissent une piste pour construire une variable d'exposition au Covid-19 par catégorie socioprofessionnelle et par secteur d'activité. Elles montrent d'abord les différences structurelles existant entre les différentes catégories : du fait de leurs conditions de travail (pénibilité, risques plus élevés...), les ouvriers et les employés sont par exemple systématiquement plus en arrêt maladie que les autres, tout comme les professionnels du secteur de la santé. C'est donc également le cas en 2020, sans que cela ne reflète nécessairement une plus forte exposition au Covid. Nous nous intéresserons donc plutôt à la différence, pour chaque secteur, entre la proportion d'actifs occupés en arrêt maladie en 2020 et celle en 2019, une différence positive reflétant donc une augmentation des arrêts maladies, que nous attribuons en partie au Covid.

Les deux tables suivantes hiérarchisent ainsi, pour les variables de catégorie socioprofessionnelle et de secteur d'activité, les catégories pour lesquelles la proportion d'arrêts de travail parmi les actifs a le plus augmenté entre 2019 et 2020.

In [ ]:
# Catégories socioprofessionnelles
exposition.exposition_diff(eec_actifs, year1 = 2019, year2 = 2020, var = 'CSE')

In [ ]:
# Secteurs d'activité
exposition.exposition_diff(eec_actifs, year1 = 2019, year2 = 2020, var = 'NAFG038UN')

Ces données permettent d'identifier les catégories parmi lesquelles les arrêts pour maladie ont le plus (ou le moins) augmenté entre 2019 et 2020. Si l'on interprète cette variation comme traduisant une plus grande exposition, la colonne `difference` oppose donc les catégories socioprofessionnelles les plus exposées au virus (chauffeurs, professions de la santé et du travail social, employés civils et agents de services de la fonction publique) aux plus protégées (clergé, grands agriculteurs, professions des arts et du spectacle), opposition que l'on retrouve par secteur.

Bien que les logiques des nomenclatures de ces deux variables peuvent se recouper (le secteur agricole est par exemple bien identifiable dans les deux cas), elles restent largement indépendantes l'une de l'autre. Cela permet d'identifier, parmi une catégorie socioprofessionnelle (les ouvriers non qualifés de l'industrie par exemple), quels secteurs économiques (transport, métallurgie, raffinage, ...) sont les plus exposés (et inversement). Pour cette raison, nous avons choisi d'utiliser simultanément les deux variables dans la construction de notre indicateur. 

Pour cela, nous découpons les modalités de ces deux variables telles qu'elles sont hiérarchisées dans les tables ci-dessus en tertiles, et attribuons un score de 2 au tertile le plus exposé, 0 au moins exposé et 1 aux catégories intermédiaires. Chaque couple de modalité `PCS x NAF`, est ainsi classé sur une échelle de 0 à 4.

In [ ]:
# Ajout de la variable de score d'exposition à la table eec_actifs
eec_actifs = exposition.score_exposition(eec_actifs, var_list = ['CSE', 'NAFG038UN'])

# Création d'une table pour l'année 2020
eec20 = eec_actifs[eec_actifs['ANNEE'] == 2020]

Une fois cette opération effectuée, il est possible de classer la population d'actifs en emploi selon leur score d'exposition. Sans surprise, les professions intermédiaires des activités pour la santé humaine obtiennent un score d'exposition de 4. Au contraire, les artisans de la construction dont l'activité a été suspendue pendant la pandémie obtiennent un score d'exposition de 0.

In [ ]:
# Groupes les plus exposés
resultat = eec20[['CSER_label', 'CSE_label', 'NAF_label', 'score_exposition']].sort_values(by='score_exposition', ascending=False)
print(resultat.head(3))

In [ ]:
# Groupes les moins exposés
resultat = eec20[['CSER_label', 'CSE_label', 'NAF_label', 'score_exposition']].sort_values(by='score_exposition', ascending=True)
print(resultat.head(3))

Le graphique suivant représente la distribution de cette variable pour l'année 2020.

In [ ]:
exposition.plot_score_exposition(eec20)

On remarque que les effectifs ne sont pas répartis de manière homogène entre les différentes catégories. Les actifs les plus exposés représentent 20,9 % de la population, tandis que les moins exposés sont seulement 7,7 %.

On peut enfin s'intéresser aux inégalités d'exposition selon différentes variables, ce qui permet déjà de présenter quelques résultats tout en vérifiant la cohérence de notre indicateur par rapport à la littérature existante. Le premier graphique représente la proportion de femmes et d'hommes pour chaque niveau d'exposition, et met clairement en évidence la part croissante de femmes à mesure que l'exposition augmente. Si certains mettent l'accent sur une potentielle plus grande vulnérabilité biologique des femmes face au Covid-19, ces résultats montrent surtout leur surreprésentation dans les métiers à forte exposition (secteurs de la santé et du care, services essentiels au public, postes en *front-office*, moindre possibilité de télétravail...)

In [ ]:
exposition.plot_freq_exposition(
    eec20,
    by='SEXE_label',
    title="Répartition de l'exposition au Covid-19 selon le sexe"
    )

Le graphique suivant présente le même graphique pour les catégories socioprofessionnelles agrégées. Il montre que, bien que les ouvriers soient plus fréquemment en arrêt, les catégories les plus exposées en 2020 sont les catégories de salariés les plus féminisées : les employées représentent 55 % et les professions intermédiaires 37 % de la catégorie la plus exposée. Les indépendants (agriculteurs exploitants et artisans, commerçants et chefs d'entreprise), avec les cadres, sont les catégories les plus protégées. Les agriculteurs en particulier, alors qu'ils ne représentent qu'environ 3 % de la population active, constituent 8 % du groupe le moins exposé.

In [ ]:
exposition.plot_freq_exposition(
    eec20,
    by='CSER_label',
    title="Répartition de l'exposition au Covid-19 selon la CSP"
    )

L’agrégation et la classification du score en tertiles peuvent cependant créer des effets de seuil accentuant certaines différences de composition entre catégories socio-professionnelles. On peut d'ailleurs rappeler ici que notre score d'exposition est calculé *relativement* aux autres catégories, et n'utilise pas de seuils fixés d'exposition, qui nous semblaient arbitraires. Enfin, nous pouvons utiliser une dernière variable commune à l'ensemble des années de la base, celle du caractère plus ou moins public de l'emploi. Ce dernier graphique rejoint les résultats précédents en soulignant la forte surreprésentation du secteur public (Etat, Collectivités locales, hôpitaux publics) dans les catégories les plus exposées. 

In [ ]:
exposition.plot_freq_exposition(
    eec20,
    by='PUB_label',
    title="Répartition de l'exposition au Covid-19 selon le statut d'emploi"
    )
# Exclues du graphique personnes non actives occupées et actifs occupés indépendants STAT2 distinct de 2

<a id="regression"></a>

### Analyse des déterminants de l'exposition au Covid-19

Comment ces trois variables explicatives - le sexe, la PCS, et le caractère public ou non de l'emploi - influencent-elles, indépendamment les unes des autres, le score d’exposition au Covid associé aux combinaisons de secteurs d’activité ?  Afin de résumer et de comparer les effets propres de ces variables explicatives, nous estimons un modèle de régression linéaire du score d’exposition. Bien que ce score soit borné entre 0 et 4, le recours à un modèle linéaire (plutôt qu'un modèle logit ordonné par exemple) demeure pertinent au regard de notre objectif principal, qui consiste à expliquer et comparer l’ampleur des effets des variables explicatives, plutôt qu’à prédire précisément le niveau du score d’exposition. 

In [ ]:
# On vérifie que les variables qu'on souhaite utiliser comme régresseurs soient bien catégorielles
categorical_vars = ["SEXE_label", "CSER_label", "PUB_label"]
# On choisit nos modalités de référence
reference_dict = {
    "SEXE_label": "Homme",
    "CSER_label": "Cadres et professions intellectuelles supérieures",
    "PUB_label": "Secteur privé"
    }
# On régresse le score d'expostion sur nos variables explicatives à l'aide d'une fonction
df_coef, modele_robust = exposition.regression_exposition(
    eec20,
    categorical_vars=categorical_vars,
    reference_dict=reference_dict
    )
# print(modele_robust.summary()) # décommenter ce code pour voir les résultats précis de la régression


In [ ]:
# Représenter graphiquement et succintement les coefficients de la régression
exposition.plot_regression_exposition(df_coef)

L'estimation des paramètres du modèle linéaire permet d'identifier les principaux facteurs explicatifs de l'exposition au Covid-19. La constante du modèle est de 1,2 et correspond au score moyen d'exposition de la catégorie de référence (des hommes cadres dans le secteur privé) sur une échelle de 0 à 4. À caractéristiques comparables, l'ensemble des modalités introduites dans le modèle se traduit par une augmentation statistiquement significative du score d'exposition.

Le coefficient associé au sexe féminin s'élève à + 0,197, indiquant une exposition résiduelle plus élevée chez les femmes que chez les hommes. Il convient toutefois de noter que l'ampleur de cet effet est relativement modeste en comparaison des écarts observés dans les statistiques descriptives brutes. Cette atténuation suggère qu'une part substantielle de la disparité de genre en matière d'exposition s'explique par la ségrégation horizontale du marché du travail (secteurs d'activité) et le type d'emplois occupés.

La hiérarchie des qualifications apparaît également comme un facteur explicatif majeur de l'exposition au Covid-19. Nous observons une corrélation inverse entre le niveau de qualification et l'exposition : par rapport aux cadres supérieurs, les employés affichent un surcroît d'exposition de 1,33 point, tandis que les ouvriers et les professions intermédiaires présentent un coefficient identique de +0,85. Ces résultats soulignent la vulnérabilité accrue des catégories d'exécution, souvent contraintes à une présence physique sur le lieu de travail.

L'appartenance au secteur public constitue un facteur de risque, en particulier dans le secteur hospitalier dont le coefficient atteint + 1,44. Ce résultat confirme le rôle de première ligne joué par les structures de soin en 2020.

D'un point de vue statistique, le modèle affiche un ajustement particulièrement élevé pour des données de santé publique, avec un $R^2$ de 0,40. Le fait que 40 % de la variance du score d'exposition soit captée par ce nombre restreint de variables témoigne de la forte dimension prédictive des déterminants socio-professionnels classiques. Toutefois, bien que ces variables soient hautement significatives, les corrélations identifiées ici ne sauraient être interprétées de manière strictement causale. Afin de pallier les limites de cette approche descriptive et de mieux isoler l'impact propre de la pandémie, la section suivante mobilise une stratégie d'identification plus robuste s'appuyant sur des méthodes quasi-expérimentales.

<a id="modelisation"></a>

## Covid long et réduction de l'offre de travail

<a id="doubles_differences"></a>

### Doubles-différences

Afin d'évaluer l'impact de la pandémie sur les différents secteurs économiques, nous mobilisons une stratégie de différences-en-différences (DiD) en comparant les secteurs exposés au Covid-19 aux secteurs non exposés. Nous estimons dans un premier temps la spécification suivante :
$$
Y_{it}=\alpha + \tau_\text{DiD}(\mathbf{1}\{t\geq 2020\}\times\mathbf{1}\{\text{exposé}_i=1\}) + \gamma_i + \delta_t + \varepsilon_{it}
$$
où $Y_{it}$ représente le niveau d’activité (ou de participation) du secteur $i$ à la période $t$. La variable $\text{exposé}_i$ est une indicatrice valant 1 si le secteur appartient au groupe traité, et $\text{Post}_t$ est une indicatrice valant 1 pour les années postérieures à 2020. Les termes $\gamma_i$ et $\delta_t$ désignent respectivement des effets fixes de secteur et de période, capturant les chocs temporels globaux et l'hétérogénéité sectorielle qui ne varie pas avec le temps. Afin de garantir des inférences statistiques valides en présence d'une potentielle autocorrélation des erreurs au sein de chaque secteur d'activité, nous utilisons des erreurs standard robustes au niveau de chaque secteur.

Notre hypothèse d’identification suppose que, hormis l'épisode d’absence pour raisons de santé en 2020, il n’existerait aucune différence moyenne de taux de participation entre les secteurs, une fois que l'on a contrôlé pour le secteur et la période (effets fixes $\gamma_i$ et $\delta_t$). Dans ce cadre, le terme $\tau_\text{DiD}$ capture l'effet moyen du traitement sur les traités (*average treatment effect*), soit l'effet propre de l'exposition au Covid-19 sur la participation au marché du travail des salariés du groupe $i$ après 2020 en moyenne.

Dans notre stratégie d'identification, nous utilisons une approche au niveau de chaque groupe $i$ construit par l'intersection d'un secteur économique NAF et d'une catégorie socio-professionnelle.

In [ ]:
# Agrégation par CSP x NAF de la proportion d'individus ayant déclaré un arrêt maladie
## Création de la variable "groupe" en combinant CSE et NAFG038UN
eec_actifs['groupe'] = eec_actifs.groupby(['CSE', 'NAFG038UN']).ngroup()

## Création d'une indicatrice d'arrêt maladie
eec_actifs['is_RABS_2'] = (eec_actifs['RABS'] == '2')

Néanmoins, certaines de ces combinaisons $i$ ont peu de sens ou ne comportent que peu d'individus, voire aucun pour certaines années. Par exemple, notre base de données comporte un groupe constitué d'ouvriers agricoles travaillant dans le secteur des industries extractives.

In [ ]:
## Tableau des effectifs inférieurs à 20 non conservés
check = eec_actifs.groupby(['groupe']).filter(
    lambda x: (x.groupby('ANNEE').size() < 20).all()
    )

check.head()

Nous agrègeons donc les données au niveau de chaque groupe $i$ pour ne conserver que ceux qui comptent plus de vingt actifs pour toutes les années de la période d'analyse. Nous obtenons 419 groupes, un échantillon de taille raisonnable pour appliquer la méthode des doubles-différences.

In [ ]:
df_dids = dids.data_prep(eec_actifs, min_obs=20)

Pour pouvoir identifier un effet, un échantillon de taille raisonnable ne suffit pas. Il est nécessaire de disposer de variabilité dans les données selon les groupes constitués et les années. On représente donc un histogramme de la proportion d'arrêts maladie par groupe $i$ et année $t$.

In [ ]:
# On regarde la distribution des arrêts maladie
plt.figure(figsize=(10, 4))
sns.histplot(df_dids['sh_arret'], bins=30, color='#7A57FF', )
plt.title('Distribution de la proportion d\'arrêts maladie (sh_arret)')
plt.xlabel('Proportion d\'arrêts')
plt.ylabel('Nombre de groupes x année (Fréquence)')
plt.show()

In [ ]:
# Calcul des statistiques
moyenne = df_dids['sh_arret'].mean()
ecart_type = df_dids['sh_arret'].std()
premier_decile = df_dids['sh_arret'].quantile(0.1)
dernier_decile = df_dids['sh_arret'].quantile(0.9)

# Affichage des résultats
print(f"Moyenne : {moyenne}")
print(f"Écart-type : {ecart_type}")
print(f"Premier décile (D1) : {premier_decile}")
print(f"Dernier décile (D9) : {dernier_decile}")

En moyenne, chaque combinaison de groupe et d'année comporte 2,8 % d'actifs en arrêt maladie. Le minimum est de 0 % et le maximum de 5,9 % pour un écart-type de 2,5 %.

On peut finalement préparer les données pour l'analyse. Pour estimer la spécification $Y_{it}=\alpha + \tau_\text{DiD}(\mathbf{1}\{t\geq 2020\}\times\mathbf{1}\{\text{exposé}_i=1\}) + \gamma_i + \delta_t + \varepsilon_{it}$, il est nécessaire de construire la variable $\mathbf{1}\{t\geq 2020\}$, propre à la spécification en doubles-différences, et la variable $\mathbf{1}\{\text{exposé}_i=1\}$ pour chaque traitement que l'on souhaite estimer, et qui sera utilisée à la fois pour les doubles-différences et l'étude d'évènement.

In [ ]:
# Préparation des données pour l'analyse
## Construction des variables d'exposition 1{exposé == 1}
print(df_dids['cat_expose'].unique())
df_dids['expose_1'] = df_dids['cat_expose'].isin([4]).astype(int)
df_dids['expose_2'] = df_dids['cat_expose'].isin([3, 4]).astype(int)
df_dids['expose_3'] = df_dids['cat_expose'].isin([2, 3, 4]).astype(int)

In [ ]:
# Données propres aux doubles-différences
## Création de la variable Post = 1{t >= 2020}
df_dids['post'] = (df_dids['ANNEE'] >= 2020).astype(int)

## Création du terme d'interaction 1{t >= 2020} x 1{exposé == 1} pour chaque définition du traitement
df_dids['traitement_1'] = df_dids['post'] * df_dids['expose_1']
df_dids['traitement_2'] = df_dids['post'] * df_dids['expose_2']
df_dids['traitement_3'] = df_dids['post'] * df_dids['expose_3']

On vérifie à présent la structure des données obtenues.

In [ ]:
# Contrôle des données
df_reg = df_dids.set_index(['groupe', 'ANNEE'])
print(f"Dimension des données : {df_reg.shape}")
print(f"Nombre de groupes uniques : {df_reg.index.get_level_values('groupe').nunique()}")
print(f"Nombre d'années uniques : {df_reg.index.get_level_values('ANNEE').nunique()}")

n_nans = df_reg[['sh_arret', 'traitement_1']].isna().sum().sum()
print(f"Nombre de valeurs manquantes : {n_nans}")

Une fois les données structurées convenablement, on peut estimer notre spécification en double-différences pour notre traitement principal `traitement_1`. Celui-ci consiste à se trouver dans le premier groupe d'exposition au Covid-19 en 2020, tandis que notre groupe de contrôle est constitué des quatre autres groupes d'exposition au Covid-19.

In [ ]:
dids.estimation_statique(df_dids, traitement='traitement_1', tableau=True)

L'estimation de la spécification en doubles-différences permet d'évaluer l'effet propre de l'exposition au Covid-19 sur la variable de participation `sh_arret`. Le coefficient associé à la variable de traitement (`traitement_1`), qui correspond au terme $\tau_\text{DiD}$ de l'équation théorique, est estimé à 0,0085. Cet effet est hautement significatif au seuil de 1 %, avec une p-valeur de 0,0009 (inférieure à 0,01) et une statistique de Student ($t$) de 3,33. L'utilisation d'erreurs standards robustes clusterisées au niveau du secteur renforce la fiabilité de cette inférence face à l'autocorrélation potentielle des résidus. **Toutes choses égales par ailleurs, l'exposition à la pandémie a entraîné une augmentation moyenne de 0,85 point de pourcentage de la proportion d'arrêts maladie pour les groupes exposés par rapport aux groupes de contrôle après 2020**.

L'estimation confirme une divergence significative des trajectoires entre les groupes exposés et non exposés suite au choc de 2020. On peut conclure que la crise sanitaire a eu un effet haussier structurel et significatif sur le recours aux arrêts de travail dans les secteurs les plus exposés.

In [ ]:
# Calcul de la moyenne pour le groupe traitement_1 == 1 et l'année 2024
moyenne_cible = df_dids[(df_dids['traitement_1'] == 1) & (df_dids['ANNEE'] == 2024)]['sh_arret'].mean()

print(f"La moyenne de sh_arret pour le groupe traité en 2024 est : {moyenne_cible:.4f}")

Au regard de la moyenne globale de l'échantillon à 2,8 % pour la variable `sh_arret`, et de 4,8 % pour le groupe le plus exposé, l'effet estimé de 0,85 point de pourcentage est particulièrement important. Il représente une augmentation relative d'environ un tiers du recours aux arrêts maladie par rapport à la moyenne de l'échantillon ($0,0085/0,028\approx30\%$), et un cinquième de la hausse du recours aux arrêts maladie par rapport à la moyenne du groupe le plus exposé ($0,0085/0,0484\approx18\%$). Cette hausse massive indique que l'impact de la pandémie n'est pas marginal, mais qu'il a profondément modifié le niveau de participation au travail dans les secteurs exposés.

Pour confirmer nos résultats, on vérifie leur robustesse à des définitions alternatives de l'exposition au Covid-19.

In [ ]:
dids.estimation_statique(df_dids, traitement='traitement_2', tableau=False)
dids.estimation_statique(df_dids, traitement='traitement_3', tableau=False)

Lorsque l'on considère dans notre groupe de traitement les groupes $i$ constitués de ceux qui se trouvent dans les deux catégories les plus exposées, on obtient un coefficient estimé de 0,46 points de pourcentage, et de 0,45 points de pourcentage si on y inclut les groupes parmi les trois catégories les plus exposées. Ces deux coefficients sont significatifs au seuil de 1 %. Nos résultats sont donc robustes à des définitions alternatives du traitement.

Les résultats obtenus par la méthode des doubles-différences sont valides sous l'hypothèse de tendances parallèles : en l'absence de traitement, les deux groupes auraient évolué de la même manière. Pour vérifier cette hypothèse et mieux saisir les effets dynamiques de l'exposition au Covid-19, nous nous tournons à présent vers la méthode de l'étude d'évènement.

<a id="event_study"></a>

### Étude d'évènement

Pour explorer la dynamique temporelle des effets et tester la validité de notre hypothèse d'identification, nous généralisons l'approche précédente par une étude d'événement (*event study*) :
$$
Y_{it} = \alpha + \sum_{\substack{j \in T \\ j \neq -1}} \tau_j (\mathbf{1}\{t=j\} \times \mathbf{1}\{\text{exposé}_i=1\}) + \gamma_i + \delta_t + \varepsilon_{it}
$$
où $\mathbf{1}\{t=j\}$ est une indicatrice égale à 1 pour l'année relative $j\in T=\{-10,\cdots,+4\}$ de sorte que les coefficients $\tau_j$, pour $j \geq 0$, capturent les effets dynamiques de l'exposition au Covid-19 $j$ période(s) après 2020. Nous fixons l'année 2019 comme période de référence, de sorte que $\tau_{-1}=0$.

Cette spécification dynamique permet de vérifier l'hypothèse fondamentale de tendances parallèles. Celle-ci postule qu'en l'absence de pandémie, l'évolution de la variable $Y_{it}$ aurait été identique pour les secteurs exposés et non exposés. La nullité et la non-significativité des coefficients $\tau_j$ pour $j \leq -2$ constitue un test de cette hypothèse.

Enfin, il convient de noter que notre cadre d'analyse repose sur une expérience naturelle où le traitement est administré de manière synchrone (en 2020). L'absence de décalage temporel dans l'adoption du traitement (staggered adoption) nous permet de nous affranchir des biais soulignés par Goodman-Bacon ([2021](https://www.sciencedirect.com/science/article/abs/pii/S0304407621001445)). Par conséquent, l'estimateur par effets fixes bidirectionnels (TWFE) demeure convergent et ne nécessite pas le recours aux estimateurs robustes développés récemment pour les traitements échelonnés (e.g., Callaway et Sant’Anna, [2021](https://www.sciencedirect.com/science/article/abs/pii/S0304407620303948) ; de Chaisemartin et d'Haultfœuille, [2022](https://www.nber.org/papers/w29873); Borusyak et al., [2024](https://academic.oup.com/restud/article/91/6/3253/7601390)).

On commence par préparer les données en construisant les indicatrices $\mathbf{1}\{t=j\}$ et $\mathbf{1}\{\text{exposé}_i=1\}$

In [ ]:
# Données propres aux études d'évènement
## Création de la variable relative j "annee_relative" centrée sur 2020
df_dids['annee_relative'] = (df_dids['ANNEE'] - 2020).astype(int)

## Création des indicatrices 1{t == j}, sauf pour l'année relative de référence j == -1
df_dids = (
    pd.get_dummies(df_dids, columns=['annee_relative'], prefix='ttt')
    # On remplace le tiret '-' par 'm' dans les noms de colonnes
    .rename(columns=lambda x: x.replace('-', 'm'))
    # On supprime la colonne de référence (ttt_m1 correspond à j = -1)
    .drop(columns='ttt_m1', errors='ignore')
)

On estime ensuite notre spécification dynamique. Pour rendre la lecture des résultats plus lisible, on les représente sous forme graphique. Chacun des points représente un $\tau_j$, et est associé à son intervalle de confiance à 95 %.

In [ ]:
# Estimation
resultat = dids.estimation_dynamique(df_dids)

# Représentation graphique
dids.plot_event_study(resultat)

Les résultats obtenus valident notre hypothèse de tendances parallèles (*pre-trends*). En effet, les coefficients associés aux périodes précédant le traitement ($j\in T=\{-10,\cdots,-2\}$) sont globalement proches de zéro et statistiquement non significatifs au seuil de 5 %. L'absence de divergence systématique entre le groupe exposé et le groupe de contrôle avant 2020 permet d'exclure l'existence de tendances préexistantes ou d'effets d'anticipation qui auraient pu biaiser l'estimation. Cette stabilité pré-traitement renforce la crédibilité de notre hypothèse d'identification : en l'absence de choc sanitaire, les deux groupes auraient suivi des trajectoires de participation au travail comparables.

Les résultats mettent aussi en lumière l'impact dynamique et la persistance de l'effet lié à l'exposition au Covid-19 en 2020 pour les coefficients associés aux périodes suivant le traitement ($j\in T=\{0,\cdots,+4\}$). Dès l'année 2020 ($j=0$), nous observons une rupture nette et statistiquement significative dans le recours aux arrêts maladie pour les secteurs exposés Le coefficient à $j=0$ s'élève à environ 0,013, marquant une augmentation immédiate de 1,3 point de pourcentage de la part des arrêts maladie. Cet effet est mécanique, puisque nous construisons notre variable de traitement à partir de l'exposition au Covid-19 en 2020.

Toutefois, les coefficients estimés atteignent leur maximum en 2021 ($j=1$), avec un coefficient avoisinant 0,014. Au regard de la moyenne de l'échantillon (2,8 %), cet effet représente une hausse relative massive de 50 % du taux d'arrêt de travail au sein des secteurs traités par rapport au groupe de référence.

Bien que l'on observe une légère érosion de l'effet à partir de $j=2$, les coefficients demeurent positifs et significatifs pour la majorité des périodes post-traitement, notamment à $j=3$. Cette persistance suggère que le choc de 2020 n'a pas été un phénomène purement transitoire, mais qu'il a induit un changement structurel dans le comportement d'absence pour raison de santé dans les secteurs les plus exposés.

L'absence de décalage temporel dans l'adoption du traitement (*staggered adoption*), celui-ci étant administré de manière synchrone en 2020, permet de confirmer la convergence de notre estimateur par effets fixes bidirectionnels (TWFE). Les résultats obtenus via cette étude d'événement corroborent l'effet moyen de 0,85 point de pourcentage identifié dans la spécification statique, tout en révélant une hétérogénéité temporelle marquée par un choc initial intense suivi d'une persistance durable de l'effet.

Comme pour l'étude en doubles-différences, il est essentiel de vérifier la robustesse de nos résultats à des définitions alternatives du traitement. Les résultats sont présentés ci-dessous et corroborent nos résultats principaux.

In [ ]:
# Traitement 2
resultat = dids.estimation_dynamique(df_dids, var_traitement='expose_2')
dids.plot_event_study(resultat)

In [ ]:
# Traitement 3
resultat = dids.estimation_dynamique(df_dids, var_traitement='expose_3')
dids.plot_event_study(resultat)

<a id="conclusion"></a>

## Conclusion

Ce projet analyse l’impact de l’exposition au Covid-19 en 2020 sur l’offre de travail de long terme des actifs en France. En exploitant les données en accès-libre de l’Enquête Emploi en Continu sur la période 2010–2024, nous mettons en évidence une augmentation durable des arrêts de travail pour raisons de santé dans les secteurs et catégories socioprofessionnelles les plus exposés au choc sanitaire de 2020.

Faute de données sur les contaminations au Covid-19 (ou diagnostics de Covid long), nous avons proposé une stratégie indirecte reposant sur l’utilisation des arrêts maladie en 2020 comme proxy de l’exposition au virus. Cette approche est confortée empiriquement par la corrélation entre arrêts maladie et hospitalisations liées au Covid-19 en 2020, corrélation qui disparaît dans les années suivantes, suggérant un changement de nature du lien entre pandémie et absences au travail après la phase aiguë de la crise sanitaire. Faute de données individuelles également, un des enjeux de ce projet a été de reconstituer des unités statistiques que l'on peut suivre dans le temps, ici la combinaison de la catégorie socio-professionnelle et du secteur d'activité. 

À partir de cette mesure d’exposition sectorielle, nos estimations en doubles-différences et en étude d’événement mettent en évidence un effet causal robuste de l’exposition au Covid-19 sur la persistance des arrêts maladie après la crise sanitaire. Les groupes les plus exposés en 2020 présentent une probabilité supérieure d’environ 0,85 point de pourcentage d’être en arrêt maladie dans les années suivantes, soit une augmentation relative d’environ un tiers par rapport à la moyenne de l’échantillon. L’analyse dynamique révèle que cet effet atteint un maximum en 2021, avant de décroître lentement tout en restant significatif jusqu’en fin de période, suggérant un choc sanitaire durable plutôt qu’un simple phénomène transitoire.

Ces résultats sont donc compatibles avec l’hypothèse selon laquelle le Covid long a contribué à une réduction persistante de l’offre de travail dans certains secteurs de l’économie française, même si nous n'avons pas pu tester directement l'effet de cette variable. Plusieurs limites importantes peuvent d'ailleurs être relevées concernant ce projet. 

Premièrement, notre méthode repose sur un proxy agrégé de l’exposition individuelle au Covid-19. Les arrêts maladie (d'ailleurs mélangé avec les accidents de travail) en 2020 ne constituent qu’une mesure imparfaite de la contamination, et encore moins du développement ultérieur de formes de Covid long. Ils peuvent également refléter d’autres mécanismes concomitants à la pandémie (changement de comportements, plus grande tolérance institutionnelle aux arrêts maladie, stress lié à la crise...). On ne peut donc pas affirmer que l'augmentation des arrêts maladies en 2020 est uniquement attribuable au Covid-19, ni de même, que l'évolution post-2020 redeviendrait indépendante de la pandémie.

Ensuite, l'absence de données individuelles longitudinales empêche d'identifier des trajectoires personnelles et de distinguer précisément les effets de l'exposition en dehors des résultats agrégés, pouvant parfois différer (voire s'inverser, cf paradoxe de Simpson). L’accès à des données administratives couplant informations sanitaires et trajectoires professionnelles, ou à des enquêtes incluant des mesures directes du Covid long, constituerait une extension naturelle et nécessaire de ce travail pour en conforter les conclusions. 

Enfin, notre variable d’intérêt - la proportion d’actifs en arrêt maladie - ne mesure pas directement l’offre de travail au sens économique strict, mais une forme très précise de non-participation temporaire pour raison de santé. Les résultats doivent donc être interprétés comme une augmentation durable de l’absentéisme pour raison déclarée de santé, et non comme une sortie définitive du marché du travail. D'autres variables étaient disponibles dans la base pour approcher l'offre de travail, telles que la non disponibilité pour travailler, la non disponibilité pour travailler davantage, la non recherche d'emploi, ou bien le temps partiel (toujours en lien avec des raisons de santé). Si des contraintes de temps et de simplicité méthodologique nous ont poussé à étudier l'impact d'une seule variable, reproduire notre projet pour les autres (voire les étudier dans leur ensemble) pourrait enrichir les analyses macro-économiques sur l'offre totale de travail. 

En dépit de ces limites, notre analyse permet de contourner certaines limites liées à l'absence de données et met en évidence l’existence d’un choc sanitaire persistant affectant l’offre de travail dans les secteurs les plus exposés à la pandémie. Elle souligne l’importance de prendre en compte les conséquences de long terme des crises sanitaires dans l’analyse du marché du travail et ouvre des perspectives de recherche sur l’articulation entre santé, conditions de travail et participation à l’emploi.
 

